In [5]:
import os
import re
from collections import defaultdict
import pandas as pd

# Percorso principale dei testi
base_path = r"F:\EPICODE\DESKTOP PUGLIA\ESAME FINALE LOOKER STUDIO\Testi_brani_Sanremo_1951-2025"

# Funzione di pulizia e tokenizzazione
def clean_and_tokenize(text):
    text = text.lower()
    text = re.sub(r"[^\w\s]", "", text)
    return text.split()

# Inizializzazione strutture dati
song_data = []

# Itera su tutte le cartelle anno
for year_folder in os.listdir(base_path):
    if year_folder.startswith("Testi_Sanremo_"):
        year = int(year_folder.split("_")[-1])
        year_path = os.path.join(base_path, year_folder)

        for file in os.listdir(year_path):
            if file.endswith(".txt"):
                file_path = os.path.join(year_path, file)
                try:
                    with open(file_path, "r", encoding="utf-8") as f:
                        text = f.read()
                except UnicodeDecodeError:
                    with open(file_path, "r", encoding="latin-1") as f:
                        text = f.read()
                
                words = clean_and_tokenize(text)
                word_count = len(words)
                unique_word_count = len(set(words))
                lexical_diversity = unique_word_count / word_count if word_count else 0

                song_data.append({
                    "anno": year,
                    "titolo": file.replace(".txt", ""),
                    "parole_totali": word_count,
                    "parole_uniche": unique_word_count,
                    "diversita_lessicale": round(lexical_diversity, 3)
                })

# Creazione DataFrame
df = pd.DataFrame(song_data)

# Salvataggio XLSX per Looker Studio
output_xlsx = os.path.join(base_path, "analisi_lessico_sanremo.xlsx")
df.to_excel(output_xlsx, index=False)


print("✅ Analisi completata. File salvato in:", output_csv)


✅ Analisi completata. File salvato in: F:\EPICODE\DESKTOP PUGLIA\ESAME FINALE LOOKER STUDIO\Testi_brani_Sanremo_1951-2025\analisi_lessico_sanremo.xlsx


In [25]:
import os
import re
from collections import Counter
import pandas as pd

# Percorso base dei testi
base_path = r"F:\EPICODE\DESKTOP PUGLIA\ESAME FINALE LOOKER STUDIO\Testi_brani_Sanremo_1951-2025"

# Funzione per pulire e tokenizzare il testo
def clean_and_tokenize(text):
    text = text.lower()
    text = re.sub(r"[^\w\s]", "", text)
    return text.split()

# Dizionario semantico
semantic_categories = {
    "amore": [
        "amore", "amori", "amato", "amata", "amati", "amate", "amare", "amando", "amante", "amanti",
        "cuore", "cuori", "bacio", "baci", "abbraccio", "abbracci", "passione", "passioni",
        "innamorato", "innamorata", "innamorati", "innamorate", "desiderio", "desideri",
        "carezza", "carezze", "affetto", "affetti", "tenerezza", "tenerezze"
    ],
    "tristezza": [
        "pianto", "pianti", "lacrima", "lacrime", "triste", "tristi", "tristezza", "tristezze",
        "solitudine", "solitudini", "dolore", "dolori", "perdere", "perso", "persa", "persi", "perse",
        "addio", "addii", "vuoto", "vuoti", "rimpianto", "rimpianti", "assenza", "assenze",
        "nostalgia", "nostalgie", "disperazione", "disperazioni"
    ],
    "gioia": [
        "gioia", "gioie", "felice", "felici", "felicità", "riso", "risa", "ridere", "sorriso", "sorrisi",
        "ballare", "ballo", "balli", "festa", "feste", "speranza", "speranze", "abbraccio", "abbracci",
        "vivere", "vita", "vite", "amico", "amica", "amici", "amiche", "allegria", "allegrie",
        "entusiasmo", "entusiasmi", "euforia", "euforie"
    ],
    "guerra": [
        "battaglia", "battaglie", "guerra", "guerre", "arma", "armi", "combattere", "combattuto",
        "combattuta", "combattuti", "combattute", "morte", "morti", "morto", "morta", "odio", "odii",
        "sangue", "sangui", "violenza", "violenze", "soldato", "soldati", "nemico", "nemica", "nemici", "nemiche"
    ],
    "natura": [
        "sole", "soli", "mare", "mari", "vento", "venti", "cielo", "cieli", "terra", "terre", "fiore", "fiori",
        "notte", "notti", "alba", "albe", "pioggia", "piogge", "neve", "nevi", "montagna", "montagne",
        "bosco", "boschi", "stella", "stelle", "luna", "lune"
    ],
    "tempo": [
        "tempo", "tempi", "ieri", "oggi", "domani", "attimo", "attimi", "eterno", "eterna", "eterni", "eterne",
        "momento", "momenti", "secondo", "secondi", "giorno", "giorni", "sera", "sere", "notte", "notti",
        "tramonto", "tramonti", "alba", "albe"
    ],
    "identità": [
        "io", "me", "persona", "persone", "anima", "anime", "nome", "nomi", "esisto", "esistere", "vero", "vera",
        "veri", "vere", "maschera", "maschere", "cuore", "cuori", "sogno", "sogni", "identità",
        "sé", "se stesso", "se stessa", "sé stessi", "sé stesse"
    ],
    "famiglia": [
        "madre", "madri", "padre", "padri", "figlio", "figlia", "figli", "figlie", "fratello", "fratelli",
        "sorella", "sorelle", "casa", "case", "famiglia", "famiglie", "nonna", "nonne", "nonno", "nonni",
        "genitore", "genitori", "parenti", "parente"
    ],
    "società": [
        "gente", "popolo", "popoli", "strada", "strade", "mondo", "mondi", "libertà", "giustizia", "lavoro",
        "lavori", "politica", "politiche", "diritto", "diritti", "realtà", "società", "comunità", "cultura",
        "culture", "nazione", "nazioni"
    ],
    "religione": [
        "dio", "dei", "fede", "preghiera", "preghiere", "paradiso", "paradisi", "inferno", "inferni", "anima",
        "anime", "angelo", "angeli", "peccato", "peccati", "miracolo", "miracoli", "croce", "croci", "santo",
        "santa", "santi", "sante"
    ],
    "musica": [
        "musica", "musiche", "canzone", "canzoni", "nota", "note", "melodia", "melodie", "cantare", "cantato",
        "cantata", "cantati", "cantate", "voce", "voci", "ritmo", "ritmi", "accordo", "accordi", "silenzio",
        "silenzi", "suono", "suoni"
    ],
    "corpo": [
        "mano", "mani", "occhio", "occhi", "viso", "visi", "pelle", "pelli", "labbro", "labbra", "respiro",
        "respiri", "voce", "voci", "petto", "petti", "sguardo", "sguardi", "abbraccio", "abbracci", "corpo",
        "corpi", "cuore", "cuori"
    ]
}

# Parole positive
positive_words = {
    "amore", "amori", "abbraccio", "abbracci", "allegria", "armonia", "aurora", "bellissimo", "bellissima",
    "bene", "bontà", "brillante", "carezza", "carezze", "cielo", "conforto", "cuore", "dolcezza", "emozione",
    "entusiasmo", "estate", "felice", "festa", "fiducia", "fortune", "gioia", "giusto", "incanto", "innamorato",
    "intesa", "libertà", "luce", "meraviglia", "musica", "ottimismo", "pace", "paradiso", "passione", "positivo",
    "promessa", "risata", "riso", "salvezza", "sereno", "sogno", "sorriso", "speranza", "splendore", "tenerezza",
    "tranquillo", "trionfo", "vittoria", "vita", "gratitudine", "soddisfazione", "orgoglio", "successo", "felicità",
    "euforia", "speranzoso"
}

# Parole negative
negative_words = {
    "odio", "tristezza", "solitudine", "disperazione", "lacrima", "pianto", "perdere", "perso", "addio", "vuoto",
    "rimpianto", "assenza", "morte", "inferno", "peccato", "paura", "dolore", "bugia", "male", "malinconia",
    "angoscia", "ansia", "invidia", "gelosia", "fallimento", "sconfitta", "disprezzo", "delusione"
}

# Analisi testi
data = []

for year_folder in os.listdir(base_path):
    year_path = os.path.join(base_path, year_folder)
    if os.path.isdir(year_path):
        for filename in os.listdir(year_path):
            if filename.endswith(".txt"):
                file_path = os.path.join(year_path, filename)
                with open(file_path, "r", encoding="utf-8") as file:
                    text = file.read()
                    tokens = clean_and_tokenize(text)
                    token_counts = Counter(tokens)

                    sem_counts = {cat: sum(token_counts[word] for word in words) for cat, words in semantic_categories.items()}
                    pos_count = sum(token_counts[word] for word in positive_words)
                    neg_count = sum(token_counts[word] for word in negative_words)


for year_folder in os.listdir(base_path):
    year_path = os.path.join(base_path, year_folder)
    if os.path.isdir(year_path):
        # Estrai l'anno come numero da 'year_folder'
        match = re.search(r"\d{4}", year_folder)
        year = int(match.group()) if match else year_folder  # Se non trova l'anno, usa il nome cartella originale

        for filename in os.listdir(year_path):
            if filename.endswith(".txt"):
                file_path = os.path.join(year_path, filename)
                with open(file_path, "r", encoding="utf-8") as file:
                    text = file.read()
                    tokens = clean_and_tokenize(text)
                    token_counts = Counter(tokens)

                    sem_counts = {
                        cat: sum(token_counts[word] for word in words)
                        for cat, words in semantic_categories.items()
                    }

                    pos_count = sum(token_counts[word] for word in positive_words)
                    neg_count = sum(token_counts[word] for word in negative_words)

                    data.append({
                        "anno": year,
                        "brano": filename.replace(".txt", ""),
                        **sem_counts,
                        "positività": pos_count,
                        "negatività": neg_count
                    })

# Creazione DataFrame e salvataggio
df = pd.DataFrame(data)
df.sort_values(by=["anno", "brano"], inplace=True)
output_path = os.path.join(base_path, "Analisi_Semantica_Sanremo.xlsx")
df.to_excel(output_path, index=False)
print(f"File salvato in: {output_path}")



File salvato in: F:\EPICODE\DESKTOP PUGLIA\ESAME FINALE LOOKER STUDIO\Testi_brani_Sanremo_1951-2025\Analisi_Semantica_Sanremo.xlsx
